This document lays out the similarity between SQL and Pandas. It is handy for those familiar with one of the two and want to use the other one as a reference.

<h3>Abstract</h3>
Display of basic functions in SQL and its corresponding practice in pandas

<h3>Pandas</h3>

Pandas data structure

Series: 1 dimension array and index can be changed

DataFrame: 2 dimension table like data structure

Panel: 3 dimension array #Deprecated

In [104]:
#Dataframe Example

import pandas as pd
import numpy as np

url="https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/data/tips.csv"
tips=pd.read_csv(url)
tips.head()

#csv file is also saved locally in case the url expires
#tips.to_csv('tips.csv',index=False)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


<h3>Basic Properties<h3>

In [105]:
df=tips
#data type of columns
df.dtypes

#indexs
df.index

#return df column name index
df.columns

#each row, return array[array]
df.values

#a tuple represent the dimension of the dataframe
df.shape

(244, 7)

<h4><font color="maroon">SELECT<font></h4>
    In SQL, the SELECT is separated by comma [,]
    <br>SELECT total_bill, tip, smoker, time FROM
    <br>tips LIMIT 5;

In [106]:
df.loc[1:3,['total_bill','tip','smoker','time']]

,total_bill,tip,smoker,time
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner


In [107]:
df.iloc[1:4,[0,1,3,5]]

,total_bill,tip,smoker,time
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner


In [108]:
df.at[3,'total_bill']
df.loc[3,'total_bill']

23.68

In [109]:
df.iat[3,0]
df.iloc[3,0]

23.68

<h4><font color="maroon">WHERE<font></h4>
    In SQL, we use WHERE for filtering
    <br>SELECT * FROM
    <br>tips LIMIT 5;
    <br>在pandas中，Dataframe可以通过多种方式进行过滤，最直观的是使用布尔索引：

In [110]:
df[df['time']=='Dinner'].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


在where子句中常常会搭配and, or, in, not关键词，Pandas中也有对应的实现：

SQL：
<br>SELECT *
<br>FROM tips
<br>WHERE time='Dinner' AND tip>5
<br>LIMIT 5;

In [111]:
# AND
df[(df['time']=='Dinner')& (df['tip']>5)].head()
#OR
df[(df['time']=='Lunch') | (df['tip']>6)].head()
# IN
df[(df['total_bill'].isin([21.01,23.68,24.59]))]
#NOT IN
df[-(df['sex']=='Male')]
df[-(df['total_bill'].isin([21.01,23.68,24.59]))].head()
#IS NOT NULL
#df[df['total_bill'].isnull()==False].head()
df[df['total_bill'].notna()==True].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


<h4><font color="maroon">DISTINCT<font></h4>
    
   <font color='green'> In SQL, we use DISTINCT for filtering
    <br>SELECT DISTINCT time FROM
    <br>tips LIMIT 5;<font>
      

In [112]:
# DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)[source]
# Return DataFrame with duplicate rows removed, optionally only considering certain columns.

# Parameters:
# subset : column label or sequence of labels, optional
# Only consider certain columns for identifying duplicates, by default use all of the columns

# keep : {‘first’, ‘last’, False}, default ‘first’
# first : Drop duplicates except for the first occurrence.
# last : Drop duplicates except for the last occurrence.
# False : Drop all duplicates.
# inplace : boolean, default False
# Whether to drop duplicates in place or to return a copy

# Returns:
# deduplicated : DataFrame
tips.drop_duplicates(subset=['time'],keep='first',inplace=True)
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
77,27.20,4.00,Male,No,Thur,Lunch,4


<h4><font color="maroon">GROUP BY<font></h4>
   <font color='green'> In SQL, we use GROUP BY for grouping
    <br>SELECT sex, count(*) FROM
    <br>tips  GROUP BY sex LIMIT 5;<font>

In [113]:
tips.groupby('sex').size()

sex
Female    1
Male      1
dtype: int64

NOTE: in pandas we use size() instead of count(). count() is applied to 
all columns, returning the NOT NULL records in each column

In [114]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,1,1,1,1,1,1
Male,1,1,1,1,1,1


Gropu By result can be applied to mutiple functions. 
<br>example: see the tips of every day of the week
<br>SELECT day, AVG(tip),COUNT( * )
<br>FROM tips
<br>GROUP BY day;

In [115]:
tips.groupby('day').agg({'tip':np.mean,'day':np.size})

,tip,day
day,,
Sun,1.01,1
Thur,4.00,1


<h4><font color="maroon">JOIN<font></h4>
<br>可以使用join()或merge()执行连接。
<br>默认情况下，join()将联接其索引上的DataFrames。
<br>每个方法都有参数，允许指定要执行的连接类型(LEFT, RIGHT, INNER, FULL)或要连接的列(列名或索引)

In [116]:
df1=pd.DataFrame({'key':['A','B','C','D'],'value':np.random.randn(4)})
df2=pd.DataFrame({'key':['B','D','D','E'],'value':np.random.randn(4)})

<h5><font color="maroon">INNER JOIN<font></h5>
<br>SELECT * FROM df1 
<br>INNER JOIN df2
<br>ON df1.key=df2.key;

In [117]:
#Pandas
#merge performs an INNER JOIN by default
pd.merge(df1,df2,on='key')

,key,value_x,value_y
0,B,-1.112263,-1.431208
1,D,0.379576,-1.225543
2,D,0.379576,-0.011383


<h5><font color="maroon">LEFT OUTER JOIN<font></h5>
<br>--- show all records from df1
<br>SELECT * FROM df1 
<br>LEFT OUTER JOIN df2
<br>ON df1.key=df2.key;

In [118]:
#Pandas

pd.merge(df1,df2,on='key',how='left')

,key,value_x,value_y
0,A,-0.290718,NaN
1,B,-1.112263,-1.431208
2,C,0.823168,NaN
3,D,0.379576,-1.225543
4,D,0.379576,-0.011383


<h5><font color="maroon">RIGHT OUTER JOIN<font></h5>
<br>--- show all records from df2
<br>SELECT * FROM df1 
<br>RIGHT OUTER JOIN df2
<br>ON df1.key=df2.key;

In [119]:
#Pandas

pd.merge(df1,df2,on='key',how='right')

,key,value_x,value_y
0,B,-1.112263,-1.431208
1,D,0.379576,-1.225543
2,D,0.379576,-0.011383
3,E,NaN,0.667031


<h5><font color="maroon">OUTER JOIN<font></h5>
<br>--- show all records from df1, df2
<br>SELECT * FROM df1 
<br>OUTER JOIN df2
<br>ON df1.key=df2.key;

In [120]:
#Pandas

pd.merge(df1,df2,on='key',how='outer')

,key,value_x,value_y
0,A,-0.290718,NaN
1,B,-1.112263,-1.431208
2,C,0.823168,NaN
3,D,0.379576,-1.225543
4,D,0.379576,-0.011383
5,E,NaN,0.667031


<h5><font color="maroon">ORDER BY<font></h5>
<br>SELECT * FROM tips
<br>ORDER BY tip DESC
<br>LIMIT 10 OFFSET 5;

In [121]:
tips.nlargest(10+5,columns='tip')

,total_bill,tip,sex,smoker,day,time,size
77,27.20,4.00,Male,No,Thur,Lunch,4
0,16.99,1.01,Female,No,Sun,Dinner,2


<h5><font color="maroon">UPDATE<font></h5>
<br>UPDATE tips
<br>SET tip = tip*2
<br>WHERE tip < 2;

In [122]:
tips.loc[tips['tip']<2,'tip']*=2
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,2.02,Female,No,Sun,Dinner,2
77,27.20,4.00,Male,No,Thur,Lunch,4


<h5><font color="maroon">DELET<font></h5>
<br>DELET FROM tips
<br>WHERE tip > 9;

In [123]:
tips=tips[tips['tip']<=9]
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,2.02,Female,No,Sun,Dinner,2
77,27.20,4.00,Male,No,Thur,Lunch,4


Reference: http://m.v.qq.com/play/play.htmlcoverid=&vid=q0836f6kewx&ptag=4_6.7.0.22106_qq